 # Data Preprocessing

In [ ]:
import requests
import json
import pandas as pd
import geopandas as gpd
import os
from shapely.geometry import Point
from geoalchemy2 import Geometry, WKTElement
import psycopg2
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Date, Float, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.schema import CreateTable
from sqlalchemy.orm import relationship
from geoalchemy2 import Geometry
from sqlalchemy.ext.declarative import declarative_base
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
app_token = '2bSOlTJkWZ0e43SGvaNbY1sHz'
API_endpoint_list = [
    "https://data.cityofnewyork.us/resource/5rq2-4hqu.json?$limit=100000000",  
    "https://data.cityofnewyork.us/resource/erm2-nwe9.json?$limit=100000000"
]


# Create a 'data' directory if it doesn't exist
data_directory = 'data'
os.makedirs(data_directory, exist_ok=True)

# Corresponding filenames for each endpoint, with path to 'data' directory
file_names = [os.path.join(data_directory, "2015StreetTreesCensus_TREES.csv"), 
              os.path.join(data_directory, "311_Service_Requests.csv")]

headers = {
    'X-App-Token': app_token
}

# Processing each API endpoint separately and saving as CSV in the 'data' folder
for i, endpoint in enumerate(API_endpoint_list):
    response = requests.get(endpoint, headers=headers)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        df.to_csv(file_names[i], index=False)
        print(f"Data from {endpoint} written to {file_names[i]} in CSV format")
    else:
        print(f"Error: {response.status_code} from {endpoint}")


In [ ]:
service_requests_path = './data/311_Service_Requests.csv'
trees_census_path = './data/2015StreetTreesCensus_TREES.csv'
rent_data_path = './data/zillow_rent_data.csv'
shapefile_path = './data/nyc_zipcodes.shp'

In [ ]:
# Cleaning Shapefiles of NYC’s Zip Codes

# Load the shapefile with geopandas
gdf = gpd.read_file('./data/nyc_zipcodes.shp')
# Assuming 'ZIPCODE' and 'geometry' are the necessary columns
gdf = gdf[['ZIPCODE', 'geometry']]
# Ensure the ZIPCODE column is a string for consistency
gdf['ZIPCODE'] = gdf['ZIPCODE'].astype(str).str.zfill(5)
# Set the CRS to a common SRID if needed (e.g., SRID 4326 for WGS84)
gdf = gdf.to_crs(epsg=4326)
# Save the cleaned data back to a shapefile
gdf.to_file('./data/cleaned_nyc_zipcodes.shp')



In [ ]:
#Cleaning Historical Monthly Average Rents by Zip Code from Zillow

# Load the Zillow Rent Data
rent_data_df = pd.read_csv(rent_data_path)
# Selecting the desired columns including region, city, countyName, and the latest rent data
# Assuming the last column is the latest rent data (you might want to verify this)
latest_rent_column = rent_data_df.columns[-1]
rent_data_df = rent_data_df[['RegionName', 'City', 'CountyName', '2023-08-31', latest_rent_column]]
# Renaming columns for clarity
rent_data_df.rename(columns={'RegionName': 'zipcode', 'City': 'city', 'CountyName': 'county_name', '2023-08-31': 'Auguest2023_rent_amount',latest_rent_column: 'rent_amount'}, inplace=True)
# Ensuring the ZIP code is a string and formatted correctly
rent_data_df['zipcode'] = rent_data_df['zipcode'].astype(str).str.zfill(5)
# Saving the cleaned data
rent_data_df.to_csv('./data/cleaned_zillow_rent_data.csv', index=False)


In [ ]:
#Cleaning Historical Data from NYC Open Data on 311 Complaints
service_requests_df = pd.read_csv(service_requests_path)
service_requests_df = service_requests_df[['unique_key', 'created_date', 'complaint_type', 'incident_zip']]
service_requests_df['created_date'] = pd.to_datetime(service_requests_df['created_date'])
service_requests_df['incident_zip'] = service_requests_df['incident_zip'].astype(str).str.split('.').str[0].str.zfill(5)
service_requests_df.dropna(subset=['incident_zip'], inplace=True)
service_requests_df.to_csv('./data/cleaned_311_Service_Requests.csv', index=False)

In [ ]:
#Cleaning the 2015 Tree Census
trees_census_df = pd.read_csv(trees_census_path)
trees_census_df = trees_census_df[['tree_id', 'spc_common', 'health', 'zipcode', 'status', 'latitude','longitude']]

trees_census_df['zipcode'] = trees_census_df['zipcode'].astype(str).str.zfill(5)
trees_census_df['health'].fillna('Unknown', inplace=True)
trees_census_df.to_csv('./data/cleaned_2015StreetTreesCensus_TREES.csv', index=False)

In [ ]:
# Create a GeoDataFrame with a 'geometry' column containing Point objects
geometry = [Point(lon, lat) for lon, lat in zip(trees_census_df['longitude'], trees_census_df['latitude'])]
trees_census_gdf = gpd.GeoDataFrame(trees_census_df, geometry=geometry, crs="EPSG:4326")

# Display the GeoDataFrame
print(trees_census_gdf)

# trees_census_gdf = trees_census_gdf.drop(columns=['longitude','latitude'])

# Storing Data

In [ ]:

# Change dataframe column name to match with the schema
trees_census_gdf.rename(columns={'tree_id': 'id'}, inplace=True)
trees_census_gdf.rename(columns={'spc_common': 'specie'}, inplace=True)
trees_census_gdf.rename(columns={'zipcode': 'zip_code_id'}, inplace=True)
trees_census_gdf['geometry'] = trees_census_gdf['geometry'].apply(lambda geom: WKTElement(geom.wkt, srid=4326))
trees_census_gdf.head()

In [ ]:
# Change dataframe column name to match with the schema
service_requests_df.rename(columns={'unique_key': 'id'}, inplace=True)
service_requests_df.rename(columns={'created_date': 'date_column'}, inplace=True)
service_requests_df.rename(columns={'incident_zip': 'zip_code_id'}, inplace=True)
service_requests_df = service_requests_df[service_requests_df['zip_code_id'] != '00nan']
service_requests_df.tail()

In [ ]:
# Change dataframe column name to match with the schema
rent_data_df.rename(columns={'Auguest2023_rent_amount': 'rent_amount_aug'}, inplace=True)
rent_data_df.rename(columns={'county_name': 'county'}, inplace=True)
rent_data_df.rename(columns={'zipcode': 'zip_code_id'}, inplace=True)
rent_data_df['id'] = range(1, len(rent_data_df) + 1)
rent_data_df = rent_data_df[['id','city', 'county', 'rent_amount_aug','rent_amount','zip_code_id']]
rent_data_df.head()

In [ ]:
zipcode_df = pd.DataFrame(rent_data_df["zip_code_id"])


# Array of values to be added
new_values = trees_census_gdf['zip_code_id']
new_values_2 = rent_data_df['zip_code_id']
new_values_3 = service_requests_df['zip_code_id']


In [ ]:
# Convert the original column to a set to identify unique values
existing_values = set(zipcode_df['zip_code_id'])

# Filter new values to only include those not present in the original column
unique_new_values = [value for value in new_values if value not in existing_values]

# Add the unique values to the original column
zipcode_df = pd.concat([zipcode_df, pd.DataFrame({'zip_code_id': unique_new_values})], ignore_index=True)

In [ ]:
# Convert the original column to a set to identify unique values
existing_values = set(zipcode_df['zip_code_id'])

# Filter new values to only include those not present in the original column
unique_new_values = [value for value in new_values_2 if value not in existing_values]

# Add the unique values to the original column
zipcode_df = pd.concat([zipcode_df, pd.DataFrame({'zip_code_id': unique_new_values})], ignore_index=True)

In [ ]:
# Convert the original column to a set to identify unique values
existing_values = set(zipcode_df['zip_code_id'])

# Filter new values to only include those not present in the original column
unique_new_values = [value for value in new_values_3 if value not in existing_values]

# Add the unique values to the original column
zipcode_df = pd.concat([zipcode_df, pd.DataFrame({'zip_code_id': unique_new_values})], ignore_index=True)

In [ ]:
zipcode_df.rename(columns={'zip_code_id': 'RegionID'}, inplace=True)

In [ ]:
zipcode_df = zipcode_df[zipcode_df['RegionID'] != '00nan']
zipcode_df = pd.DataFrame(zipcode_df['RegionID'].unique())
zipcode_df.columns = ["id"]
# zipcode_df.columns = ["ZIPCODE"]

In [ ]:

def drop_table(connection_params, table_name):
    try:
        # Establish a connection to the PostgreSQL database
        conn = psycopg2.connect(**connection_params)
        conn.autocommit = True  # Set autocommit to True for DDL statements

        # Create a cursor
        cursor = conn.cursor()

        # Drop the table
        query = f"DROP TABLE IF EXISTS {table_name} CASCADE;"
        cursor.execute(query)

        print(f"Table {table_name} and its columns dropped successfully.")

    except psycopg2.Error as e:
        print(f"Error: {e}")
    finally:
        # Close the cursor and connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Specify the table name to drop
table_to_drop = 'complaints'
table_to_drop_2 = 'trees'
table_to_drop_3 = 'zip_codes'
table_to_drop_4 = 'rents'

# Call the function to drop the table
drop_table(db_params, table_to_drop)
drop_table(db_params, table_to_drop_2)
drop_table(db_params, table_to_drop_3)
drop_table(db_params, table_to_drop_4)


In [ ]:
db_params = {
    'host': 'localhost',
    'database': 'ritajkx',
    'user': 'ritajkx', 
    'password': 'none',
    'port': '5432', 
}

# Construct the connection string
conn_string = "postgresql://{user}:{password}@{host}:{port}/{database}".format(**db_params)

# Create an SQLAlchemy engine
engine = create_engine(conn_string)

Base = declarative_base()


class ZipCode(Base):
    __tablename__ = 'zip_codes'
    id = Column(Integer, primary_key=True)
#     geometry = Column(Geometry(geometry_type='POLYGON', srid=4326))
    # Relationships
    complaints = relationship("Complaint", backref="zip_code")
    trees = relationship("Tree", backref="zip_code")
    rents = relationship("Rent", backref="zip_code")

class Complaint(Base):
    __tablename__ = 'complaints'
    id = Column(Integer, primary_key=True)
    date_column = Column(Date)
    complaint_type = Column(String)
    zip_code_id = Column(Integer, ForeignKey('zip_codes.id'))

class Tree(Base):
    __tablename__ = 'trees'
    id = Column(Integer, primary_key=True)
    specie = Column(String)
    health = Column(String)
    status = Column(String)
    zip_code_id = Column(Integer, ForeignKey('zip_codes.id'))
    longitude = Column(Float)
    latitude = Column(Float)
    geometry = Column(Geometry(geometry_type='POINT', srid=4326))

class Rent(Base):
    __tablename__ = 'rents'
    id = Column(Integer, primary_key=True)
    city = Column(String)
    county = Column(String)
    rent_amount_aug = Column(Float)
    rent_amount = Column(Float)
    zip_code_id = Column(Integer, ForeignKey('zip_codes.id'))

    

# Generate schema.sql
with open('schema.sql', 'w') as f:
    for table in [ZipCode.__table__, Complaint.__table__, Tree.__table__, Rent.__table__]:
        f.write(str(CreateTable(table)))
        f.write(";\n")

# Create the tables in the database
Base.metadata.create_all(engine)

In [ ]:
# Define the table name
table_name_1 = 'zip_codes'
table_name_2 = 'trees'
table_name_3 = 'rents'
table_name_4 = 'complaints'

# Use the to_sql method to write the DataFrame to the SQL database
zipcode_df.to_sql(table_name_1, con=engine, index=False, if_exists='append')
# zipcode_gdf.to_sql(table_name_1, con=engine, index=False, if_exists='append', dtype={'geometry': Geometry('POLYGON', srid=4326)})

In [ ]:
rent_data_df.to_sql(table_name_3, con=engine, index=False, if_exists='append') 

In [ ]:
service_requests_df.to_sql(table_name_4, con=engine, index=False, if_exists='append')  

In [ ]:
trees_census_gdf.to_sql(table_name_2, con=engine, index=False, if_exists='append', dtype={'geometry': Geometry('POINT', srid=4326)})  

In [ ]:
gpd.read_postgis('trees', conn_string, geom_col='geometry')  

In [ ]:
pd.read_sql_table('zip_codes', conn_string) 

In [ ]:
pd.read_sql_table('rents', conn_string) 

In [ ]:
pd.read_sql_table('complaints', conn_string) 

# Understanding Data

Query 1*: Which area might be more calm to live in?

Between October 1st, 2022 and September 30th, 2023 (inclusive), find the number of 311 complaints per zip code. 

In [ ]:
def get_complaints_count_per_zip(engine):
    query = """
    SELECT zip_code_id, COUNT(*) AS complaint_count
    FROM complaints
    WHERE date_column BETWEEN '2022-10-01' AND '2023-09-30'
    
    GROUP BY zip_code_id
    ORDER BY complaint_count DESC;
    """
    return pd.read_sql_query(query, engine)

complaints_per_zip_df = get_complaints_count_per_zip(engine)

print(complaints_per_zip_df)

Query 2: Where has the most greenery?

Using just the trees table, which 10 zip codes have the most trees?

The query result should have two columns, 10 rows. The rows should be sorted by the total number of trees, descending.


In [ ]:
def get_top_zipcodes_with_most_trees(engine):
    query = """
    SELECT zip_code_id, COUNT(*) AS tree_count
    FROM trees
    GROUP BY zip_code_id
    ORDER BY tree_count DESC
    LIMIT 10;
    """
    return pd.read_sql_query(query, engine)

top_zipcodes_with_most_trees_df = get_top_zipcodes_with_most_trees(engine)

print(top_zipcodes_with_most_trees_df)

Query 3: Can I afford a place in the areas with the most trees?

Of the 10 zip codes with the most trees, for the month of August 2023,what is the average rent by zip code?

In [ ]:
def get_average_rent_with_most_trees(engine):
    query = """
        WITH TopZipCodes AS (
            SELECT zip_code_id, COUNT(*) AS tree_count
            FROM trees
            GROUP BY zip_code_id
            ORDER BY tree_count DESC
            LIMIT 10
        )
        SELECT TopZipCodes.zip_code_id, 
               TO_CHAR(rents.rent_amount_aug, '9999.99') AS average_rent
        FROM TopZipCodes 
        JOIN rents  ON TopZipCodes.zip_code_id = rents.zip_code_id
        ORDER BY TopZipCodes.tree_count DESC
        LIMIT 10;
    """
    return pd.read_sql_query(query, engine)

get_average_rent_with_most_trees_df = get_average_rent_with_most_trees(engine)

print(get_average_rent_with_most_trees_df)

Query 4: Could there be a correlation between an area’s rent, the number of its trees, and the number of 311 complaints?


In [ ]:
def get_five_best_and_five_worst(engine):
    query = """
    WITH RentStatistics AS (
        SELECT
            rents.zip_code_id,
            TO_CHAR(AVG(rents.rent_amount), '9999.99') AS avg_rent,
            COUNT(trees.id) AS tree_count,
            COUNT(complaints.id) AS complaint_count
        FROM rents 
        LEFT JOIN trees ON rents.zip_code_id = trees.zip_code_id
        LEFT JOIN complaints  ON rents.zip_code_id = complaints.zip_code_id
        WHERE EXTRACT(MONTH FROM complaints.date_column) = 12 AND EXTRACT(YEAR FROM complaints.date_column) = 2023
        GROUP BY rents.zip_code_id
    )
    SELECT
        zip_code_id,
        avg_rent,
        tree_count,
        complaint_count
    FROM (
        SELECT
            zip_code_id,
            avg_rent,
            tree_count,
            complaint_count,
            ROW_NUMBER() OVER (ORDER BY avg_rent ASC) AS low_rank,
            ROW_NUMBER() OVER (ORDER BY avg_rent DESC) AS high_rank
        FROM RentStatistics
    ) ranked
    WHERE low_rank <= 5 OR high_rank <= 5
    ORDER BY avg_rent ASC;
    """
    return pd.read_sql_query(query, engine)

get_five_best_and_five_worst_df = get_five_best_and_five_worst(engine)

print(get_five_best_and_five_worst_df)


Query 5: Where has the most greenery (take 2)?
Rewrite Query 2 to use both the trees table and the zipcodes table. Join both tables where the coordinate point of the tree is inside the polygon boundary of the zipcode as defined in the zipcode table.
The query should have a JOIN statement. The query results should match exactly the results of Query 2.

Query 6: What is the immediate area like?
Using the following coordinate pair on campus, which trees are within 1⁄2 mile radius of this point?
Latitude: 40.80737875669467, Longitude: -73.96253174434912
The result should have 5 columns (ID, species, health, status, and coordinate location of each tree).

In [ ]:
def immediate_area(engine):
    query = """
    SELECT
        trees.id AS ID,
        trees.specie AS species,
        trees.health,
        trees.status,
        ST_AsText(trees.geometry) AS coordinate_location
    FROM
        trees
    JOIN
        zip_codes  ON trees.zip_code_id = zip_codes.id
    WHERE
        ST_DWithin(
            trees.geometry,
            ST_SetSRID(ST_MakePoint(-73.96253174434912, 40.80737875669467), 4326),
            804.672  -- 1/2 mile in meters (1 mile = 1609.344 meters)
        );

    """
    return pd.read_sql_query(query, engine)

immediate_area_df = immediate_area(engine)

print(immediate_area_df)

# Visualizing Data

Visualization 1

In [ ]:


def select_complains(engine):
    query = """
    SELECT *
    FROM complaints;
    """
    return pd.read_sql_query(query, engine)

select_complains_df = select_complains(engine)
print(select_complains_df)

In [ ]:
# Set the date range for the analysis (October 1st, 2022, to September 30th, 2023)
start_date = '2022-10-01'
end_date = '2023-09-30'

select_complains_df['date_column'] = select_complains_df['date_column'].astype(str)

# Filter complaints for the specified date range
filtered_complaints = select_complains_df[
    (select_complains_df['date_column'] >= start_date) &
    (select_complains_df['date_column'] <= end_date)
]

# Find the top 3 complaint types
top_complaint_types = filtered_complaints['complaint_type'].value_counts().nlargest(3).index

# Filter complaints for the top 3 types
top_complaints = filtered_complaints[filtered_complaints['complaint_type'].isin(top_complaint_types)]

# Create a pivot table to count the number of complaints per day for each complaint type
complaints_pivot = top_complaints.pivot_table(index='date_column', columns='complaint_type', aggfunc='size', fill_value=0)

# Plot the data using seaborn
plt.figure(figsize=(12, 6))
sns.lineplot(data=complaints_pivot, palette='husl', linewidth=2)
plt.title('Number of Complaints per Day for Top 3 Complaint Types')
plt.xlabel('Date')
plt.ylabel('Number of Complaints')
plt.legend(title='Complaint Type')
plt.show()

Visualization 2

In [ ]:
# # Define the date range for the analysis (October 1st, 2018, to September 30th, 2023)
start_date = '2018-10-01'
end_date = '2023-09-30'


# Construct the SQL query to select complaints data for zip code 10027 within the specified date range
sql_query = f"""
    SELECT complaints.complaint_type, COUNT(*) AS complaint_count
    FROM complaints 
    JOIN zip_codes ON complaints.zip_code_id = zip_codes.id
    WHERE zip_codes.id = 10027 AND complaints.date_column BETWEEN '{start_date}' AND '{end_date}'
    GROUP BY complaints.complaint_type
    ORDER BY complaint_count DESC
    LIMIT 10;
"""

# Read data from the database into a pandas DataFrame
complaints_by_type = pd.read_sql(sql_query, engine)

# Plot the data using seaborn
plt.figure(figsize=(12, 6))
sns.barplot(x='complaint_count', y='complaint_type', data=complaints_by_type, palette='viridis')
plt.title('Top 10 Complaint Types in Zip Code 10027 (Oct 2018 - Sep 2023)')
plt.xlabel('Number of Complaints')
plt.ylabel('Complaint Type')
plt.show()


Visualization 3

In [ ]:
# Define the date range for the analysis (January 1st, 2015, to September 30th, 2023)
start_date = '2015-01-01'
end_date = '2023-09-30'

# Construct the SQL queries to select data for rent, trees, and complaints by zip code within the specified date range
sql_rent_query = f"""
    SELECT zip_codes.id, AVG(rents.rent_amount) AS average_rent
    FROM rents 
    JOIN zip_codes ON rents.zip_code_id = zip_codes.id
    GROUP BY zip_codes.id
"""

sql_trees_query = f"""
    SELECT zip_codes.id, COUNT(*) AS tree_count
    FROM trees 
    JOIN zip_codes  ON trees.zip_code_id = zip_codes.id
    WHERE trees.longitude IS NOT NULL AND trees.latitude IS NOT NULL
    GROUP BY zip_codes.id
"""

sql_complaints_query = f"""
    SELECT zip_codes.id, COUNT(*) AS complaint_count
    FROM complaints 
    JOIN zip_codes  ON complaints.zip_code_id = zip_codes.id
    WHERE complaints.date_column BETWEEN '{start_date}' AND '{end_date}'
    GROUP BY zip_codes.id
"""

# Read data from the database into pandas DataFrames
df_rent = pd.read_sql(sql_rent_query, engine)
df_trees = pd.read_sql(sql_trees_query, engine)
df_complaints = pd.read_sql(sql_complaints_query, engine)

# Merge DataFrames on 'zip_code'
df_combined = pd.merge(df_rent, df_trees, on='id', how='outer')
df_combined = pd.merge(df_combined, df_complaints, on='id', how='outer')

# Plotting
plt.figure(figsize=(15, 8))

# Subplot 1: Rent vs. Trees
plt.subplot(2, 1, 1)
sns.scatterplot(x='average_rent', y='tree_count', data=df_combined, hue='id', palette='viridis', legend=False)
plt.title('Rent vs. Number of Trees by Zip Code')

# Subplot 2: Rent vs. Complaints
plt.subplot(2, 1, 2)
sns.scatterplot(x='average_rent', y='complaint_count', data=df_combined, hue='id', palette='viridis', legend=False)
plt.title('Rent vs. Number of Complaints by Zip Code')

# Adjust layout and show the plots
plt.tight_layout()
plt.show()


Visualization 4: If I can afford more in rent, will there be fewer issues & complaints?
Create a boxplot, where the x-axis is average rent in September 2023, separated into $1000 bins (i.e. $0-1000, $1001-2000, etc), and the y-axis is the number of 311 complaints observed in each zip code between October 1, 2022 (inclusive) to September 30, 2023 (inclusive).

In [ ]:
# Define the date range for the analysis (October 1st, 2022, to September 30th, 2023)
start_date = '2022-10-01'
end_date = '2023-09-30'

# Construct the SQL query to select data for average rent and number of complaints by zip code within the specified date range
sql_query = f"""
    SELECT zip_codes.id, AVG(rents.rent_amount) AS average_rent, COUNT(complaints.id) AS complaint_count
    FROM rents 
    JOIN zip_codes  ON rents.zip_code_id = zip_codes.id
    LEFT JOIN complaints  ON zip_codes.id = complaints.zip_code_id AND complaints.date_column BETWEEN '{start_date}' AND '{end_date}'
    GROUP BY zip_codes.id
"""

# Read data from the database into a pandas DataFrame
df_combined = pd.read_sql(sql_query, engine)

# Create rent bins
rent_bins = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

# Assign rent bins to the DataFrame
df_combined['rent_bin'] = pd.cut(df_combined['average_rent'], bins=rent_bins, right=False)

# Plotting
plt.figure(figsize=(12, 6))
sns.boxplot(x='rent_bin', y='complaint_count', data=df_combined, palette='viridis')
plt.title('Boxplot: Rent vs. Number of 311 Complaints (Oct 2022 - Sep 2023)')
plt.xlabel('Average Rent (Rent Bins)')
plt.ylabel('Number of 311 Complaints')
plt.xticks(rotation=45)
plt.show()


Visualization 5: Where were the recent 311 incidents reported from in the immediate area?
Create a geospatial plot of the coordinates of reported 311 incidents that happened between January 1st, 2023 and September 30th, 2023 (inclusive) within a 1 kilometer radius of the same coordinate from Query 6 in Part 3.

In [ ]:
# Define the coordinate from Query 6
center_coordinate = Point(-73.96253174434912, 40.80737875669467)

# Define the date range for the analysis (January 1st, 2023, to September 30th, 2023)
start_date = '2023-01-01'
end_date = '2023-09-30'

# Construct the SQL query to select 311 incidents data within the specified date range and around the specified coordinate
sql_query = f"""
    SELECT
        complaints.id,
        ST_AsText(trees.geometry)::geometry AS coordinate_location
    FROM
        complaints 
    JOIN
        zip_codes  ON complaints.zip_code_id = zip_codes.id
    JOIN
        trees ON trees.zip_code_id = zip_codes.id
    WHERE
        complaints.date_column BETWEEN '{start_date}' AND '{end_date}'
        AND ST_DWithin(
            trees.geometry,
            ST_SetSRID(ST_MakePoint(-73.96253174434912, 40.80737875669467), 4326),
            1000  -- 1 kilometer in meters
        )
"""

# Read data from the database into a GeoDataFrame
gdf_complaints = gpd.read_postgis(sql_query, engine, geom_col='coordinate_location')

# Plotting
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the coordinate from Query 6
ax.scatter(center_coordinate.x, center_coordinate.y, color='red', label='Query 6 Coordinate')

# Plot the 311 incident coordinates
gdf_complaints.plot(ax=ax, marker='o', color='blue', label='311 Incidents')

# Set plot title and labels
plt.title('Geospatial Plot of 311 Incidents (Jan 2023 - Sep 2023) around Query 6 Coordinate')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Add legend
ax.legend()

# Show the plot
plt.show()


Visualization 6: Are areas trying to improve the amount of trees in the neighborhood?
Create a geospatial plot of two sets of data: the coordinates of trees in NYC, and the coordinates of "New Tree Request" 311 complaint types that were made from October 1st, 2018 to September 30th, 2023 (inclusive).

In [ ]:
# Construct the SQL query to select tree coordinates
trees_query = """
    SELECT
        id,
        ST_AsText(geometry)::geometry AS geometry
    FROM
        trees
"""

# Construct the SQL query to select "New Tree Request" 311 complaint coordinates
complaints_query = """
    SELECT
        complaints.id,
        ST_AsText(trees.geometry)::geometry AS geometry
    FROM
        complaints 
    JOIN
        zip_codes  ON complaints.zip_code_id = zip_codes.id
    JOIN
        trees ON trees.zip_code_id = zip_codes.id
    WHERE
        date_column BETWEEN '2018-10-01' AND '2023-09-30'
        AND complaint_type = 'New Tree Request'
"""


# Read data from the database into GeoDataFrames
gdf_trees = gpd.read_postgis(trees_query, engine, geom_col='geometry')
gdf_complaints = gpd.read_postgis(complaints_query, engine, geom_col='geometry')

# Plotting
fig, ax = plt.subplots(figsize=(12, 12))

# Plot the coordinates of trees
gdf_trees.plot(ax=ax, marker='o', color='green', markersize=5, label='Tree Coordinates')

# Plot the coordinates of "New Tree Request" 311 complaints
gdf_complaints.plot(ax=ax, marker='o', color='red', markersize=5, label='New Tree Request 311 Complaints')

# Set plot title and labels
plt.title('Geospatial Plot of Trees and New Tree Request 311 Complaints in NYC')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Add legend
ax.legend()

# Show the plot
plt.show()
